In [1]:
%%capture
!pip install transformers
!pip install gdown
!pip install opencc-python-reimplemented
!pip install googletrans==3.1.0a0
!pip install fake_useragent

In [2]:
def save():
  %notebook 鉅亨網文字探勘.ipynb 
  !cp 鉅亨網文字探勘.ipynb "/content/drive/MyDrive/Colab Notebooks"

In [6]:
#save()

In [3]:
import torch
import numpy as np
import pandas as pd

In [4]:
from transformers import BertModel, BertTokenizer
PATH = '/content/FinBERT_L-12_H-768_A-12_pytorch'

In [5]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent


In [6]:
import time
import datetime as dt
import time
import tqdm

In [7]:
from opencc import OpenCC
cc = OpenCC('t2s')

In [8]:
#!gdown https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
#!unzip /content/FinBERT_L-12_H-768_A-12_pytorch.zip

# 財金BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PATH ,local_files_only=True)
model = BertModel.from_pretrained(PATH ,local_files_only=True)

In [ ]:
sector_dict = {
'健護': '醫療保健服務類股',
'科技': '科技類股', 
'公用事業': '公用事業類股',
'生技':'生技類股', 
'能源':'能源類股',
'替代能源':'替代能源類股',
'消費':'消費品類股',
'金融':'金融類股', 
'礦業':'礦業',
'天然資源':'天然資源', 
'貴金屬現貨':'貴金屬',
'農產品':'農產品',
'貴金屬類股':'貴金屬',
}

In [ ]:
mkt_dict= [
'新興市場債',
'高收益債',
'全球債',
'投資等級公司債',
'美國公債',
'亞債',
'REITs不動產投資信託',
'新臺幣類貨幣',
'外幣類貨幣',
'亞太股市', '美國股市', '全球新興市場', 
'全球型股市', '大中華股市', '中國A股', '印度股市','歐洲股市', 
'東南亞股市', '拉丁美洲股市', 
'韓國股市', '新興歐洲股市', '德國股市', '台灣股市','俄羅斯股市',
'中國H股', '日本股市', '英國股市','泰國股市',
'香港股市', '澳洲股市', '印尼股市', '巴西股市', '歐非中東股市'
]


In [ ]:
mkt_dict = {x:x for x in mkt_dict}
mkt_dict['亞債'] = '亞洲債券'
mkt_dict['REITs不動產投資信託'] = '不動產'

In [ ]:
total_sec_dict = {**sector_dict, **mkt_dict}

In [ ]:
sentences = list(set(total_sec_dict.values()))
sentences = [cc.convert(x) for x in sentences]

In [ ]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
x = outputs['last_hidden_state']
np.savetxt('bert_mkt_emb.tsv', x[:,0].cpu().detach().numpy(), delimiter="\t")
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

# 鉅亨網爬蟲 初步整理


In [ ]:
res = requests.get('https://news.cnyes.com/news/cat/wd_stock')

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')

In [ ]:
%%capture

"""
news_list = soup.find_all("a", attrs={"class":"_1Zdp"})
[x['href'] for x in news_list]
[x.find('h3').get_text(strip=True) for x in news_list
"""
## .get_text(strip=True) 可避免不必用空白符亂碼

In [ ]:
ua = UserAgent()
headers = {'user-agent': ua.random}

In [ ]:
## 將資料起迄日轉換unix timestamp

start_unix_timestamp = int(time.mktime(dt.date(2021,1,1).timetuple()))
end_unix_timestamp = int(time.mktime(dt.date(2021,11,15).timetuple()))

In [ ]:
url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, 1)
res = requests.get(url_api, headers=headers)
res_json = res.json()

In [ ]:
last_page = res_json['items']['last_page']
last_page

In [ ]:
news_data_list = []

for i in tqdm.tqdm(range(1,last_page+1)):
  url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, i)
  res = requests.get(url_api, headers=headers)
  res_json = res.json()
  news_data_list.append(res_json)
  time.sleep(1)

100%|██████████| 613/613 [23:20<00:00,  2.29s/it]


In [10]:
import pickle
'''
with open('鉅亨網2021新聞.pickle', 'wb') as f:
  pickle.dump(news_data_list, f)
'''

"\nwith open('鉅亨網2021新聞.pickle', 'wb') as f:\n  pickle.dump(news_data_list, f)\n"

In [19]:
!gdown https://drive.google.com/uc?id=1ATL1IdoLgLfV-ro1r5BQzPp8DZXVqfws

Downloading...
From: https://drive.google.com/uc?id=1ATL1IdoLgLfV-ro1r5BQzPp8DZXVqfws
To: /content/___2021__.pickle
100% 61.3M/61.3M [00:00<00:00, 132MB/s] 


In [11]:
with open('/content/___2021__.pickle', 'rb') as f:
  news_data_list = pickle.load(f)

In [12]:
import re

In [13]:
## 只保留英文中文數字 

def remove_punctuation(line):
  #rule = re.compile("[^a-z A-Z \u4e00-\u9fa5]")
  rule = re.compile("[^a-z A-Z 0-9 \u4e00-\u9fa5]")
  line = rule.sub('',line)
  return line

def remove_char(line):
  rule = re.compile('&lt;.+?(&gt;|>)')
  line = rule.sub('',line)
  return line


In [14]:
from datetime import datetime
ts = int('1636969777')

## UNIX timestamp轉日期
print(dt.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-11-15 09:49:37


In [15]:
## todo:取出該新聞的相關標的

In [51]:
news_extracted_list=[]

for chunk in news_data_list:
  article_content = chunk['items']['data']
  data_dict = [{
      'id':x['newsId'], 
      'title':x['title'], 
      'summary':x['summary'],  
      'article':remove_char(x['content']),  
      #'stock':x['market'],

      'unix_timestamp':x['publishAt'],
      'datetime':dt.datetime.utcfromtimestamp(x['publishAt']).strftime('%Y-%m-%d')
      } for x in article_content
  ]
  news_extracted_list.extend(data_dict)


In [52]:
org_corpus_df = pd.DataFrame(news_extracted_list)

In [53]:
org_corpus_df['article'] = org_corpus_df['article'].str.replace('\n\n',"")
org_corpus_df['article'] = org_corpus_df['article'].str.replace(" ", "")

In [54]:
org_corpus_df['article'] = org_corpus_df['article'].str.replace(" ", "")

In [55]:
#org_corpus_df['summary'] = org_corpus_df['summary'].astype(str).apply(lambda x: remove_punctuation(x))

# 分詞 資料整理

In [56]:
text = org_corpus_df['summary'].astype(str).tolist()

In [57]:
text[0:5]

['混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 23.5 億元，出售予興富發 (2542-TW) 子公司碧江企業，處分利益達 14.9 億元，估貢獻每股純益約 1.27 元。',
 '台泥 (1101-TW) 今 (15) 日召開法說，總經理李鐘培表示，10 月以來中國能耗雙控持續，水泥產銷量減但價格持穩高檔，且近期煤價大幅回落，有利毛利率回升到過往水準之上，加上廣東韶關廠投產也將挹注營收表現。',
 '台泥 (1101-TW) 董事長張安平今 (15) 日在法說會前致詞時指出，今天的氣候災害，是人類打造繁榮的副作用漸漸浮現、失衡的大自然對人類的反撲；與人類文明與大自然息息相關的水泥業「站在生命的第一現場」。',
 '聚醯亞胺（PI）薄膜廠達邁 (3645-TW) 積極展開產品應用多元化的布局，由以往高達 98% 的軟板應用，在今年前三季的比重已降至 64%，而在非軟板的應用的成長到 36% 則以車用、散熱產品為主。',
 '22 家壽險前 3 季合賺 3389 億元破紀錄 淨值比 all pass！']

In [58]:
#re.sub(r"(?m)^\s+", "", " ")

### 停用詞

In [24]:
!wget https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt

--2021-11-22 15:32:37--  https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4717 (4.6K) [text/plain]
Saving to: ‘cn_stopwords.txt.1’

cn_stopwords.txt.1  100%[===================>]   4.61K  --.-KB/s    in 0s      

2021-11-22 15:32:38 (10.7 MB/s) - ‘cn_stopwords.txt.1’ saved [4717/4717]



In [25]:
## 逐行讀入
with open('/content/cn_stopwords.txt', 'r') as f:
  stop_words = f.read().splitlines()

In [26]:
from opencc import OpenCC
cc = OpenCC('s2t')

## 轉繁體
stop_words = [cc.convert(text) for text in stop_words]

In [27]:
## 保留數字
stop_words = list(filter(lambda x: x not in ['0','1','2','3','4','5','6','7','8','9'], stop_words))

### bert版斷詞 偏慢

In [59]:
%%capture
!pip install -U ckip-transformers
#https://github.com/ckiplab/ckip-transformers


In [60]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [30]:
ws_driver = CkipWordSegmenter(level=3,device=0)


In [62]:
corpus_ws_org  = ws_driver(text)

Inference: 100%|██████████| 66/66 [01:07<00:00,  1.02s/it]


In [ ]:
corpus_ws_org[0:5]

In [ ]:
corpus_ws_org[0:5]

### lstm斷詞

In [ ]:
!pip install -U ckiptagger[gdown]
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
data_utils.download_data_url("./")
ws = WS("./data", disable_cuda=False)

In [ ]:
%%time
corpus_ws_org  = ws(text)

CPU times: user 32.9 s, sys: 4.8 s, total: 37.7 s
Wall time: 21.9 s


### 方便套件套件處理的空白分隔模式

In [ ]:
#org_corpus_df['summary'] = org_corpus_df['summary'].astype(str).apply(lambda x: remove_punctuation(x))

In [69]:
## 刪除空白的element
corpus_ws_org = [list(filter(lambda x: x!="" and x!=" ", corpus)) for corpus in corpus_ws_org if corpus !=[]]

In [74]:
corpus_ws_org = [list(map(lambda x: remove_punctuation(x), corpus)) for corpus in corpus_ws_org if corpus !=[]]

In [75]:
## 刪除element裡面的空白

corpus_ws_rm =[]
for corpus in corpus_ws_org:
  pattern = re.compile(r'\s+')
  corpus_ws_rm.append([re.sub(pattern, '', x) for x in corpus])


In [35]:
## 把停用詞刪除
corpus_ws_rm = [[x for x in sentence if x not in stop_words] for sentence in corpus_ws_rm]

In [36]:
corpus_ws_rm[6:10]

[['台灣',
  '光罩',
  '2338',
  'TW',
  '15',
  '日',
  '公告',
  '為',
  '因應',
  '光罩',
  '業務',
  '發展',
  '組織',
  '擴展',
  '計畫',
  '增設',
  '營運',
  '副總',
  '漢磊',
  '3707',
  'TW',
  '前',
  '營運',
  '副總',
  '曾哲斌',
  '出任',
  '漢磊',
  '長期',
  '著墨',
  '化合物',
  '半導體',
  '領域',
  '光罩',
  '次',
  '派任',
  '具',
  '化合物',
  '半導體',
  '背景',
  '經理人',
  '擔任',
  '要職'],
 ['2021',
  '道瓊',
  '永續',
  '指數',
  'DJSI',
  '評比',
  '出爐',
  '東元',
  '連續',
  '兩',
  '年',
  '入選',
  '道瓊',
  '永續',
  '指數',
  'DJSI',
  '新興',
  '市場',
  '指數',
  'DJSIEmerging',
  'Markets',
  '為',
  '新興',
  '市場',
  '指數',
  '類別',
  '中',
  '全球',
  '唯一',
  '入選',
  '機電',
  '業者'],
 ['台泥',
  '1101',
  'TW',
  '15',
  '日',
  '召開',
  '法說',
  '外界',
  '關注',
  '綠能',
  '儲',
  '布局',
  '董事長',
  '張安平',
  '外',
  '揭',
  '櫫',
  '儲能',
  '子公司',
  'NHOA',
  '十',
  '倍',
  '成長',
  '10',
  'X',
  '計畫',
  '預計',
  '2030',
  '南歐',
  '充電樁',
  '市場',
  '達到',
  '15',
  '市佔率',
  '營收',
  '會',
  '2022年',
  '15',
  '倍'],
 ['檢測',
  '試劑廠',
  '瑞基',
  '4171',
  'TW',
  '15',
  '日',
  '表示',
 

In [77]:

## 將乾淨資料用空白連起來, 方便套件讀入
corpus_white_join = [" ".join(x) for x in corpus_ws_rm]

In [78]:
corpus_white_join[6:10]

['台灣 光罩 2338TW 今 15 日 公告 為 因應 光罩 業務 發展 與 組織 擴展 計畫 增設 營運 副總 由 漢磊 3707TW 前 營運 副總 曾哲斌 出任 由於 漢磊 長期 著墨 在 化合物 半導體 領域 光罩 此 次 派任 具 化合物 半導體 背景 的 經理人 擔任 要職',
 '2021 道瓊 永續 指數 DJSI 評比 出爐 東元 連續 兩 年 入選 道瓊 永續 指數 DJSI 的 新興 市場 指數 DJSIEmerging Markets 為 新興 市場 指數 類別 中 全球 唯一 入選 的 機電 業者',
 '台泥 1101TW 今 15 日 召開 法說 對於 外界 關注 的 綠能 儲能 布局 董事長 張安平 也 也 對 外 揭櫫 儲能 子公司 NHOA 的 十 倍 成長 的 10 X 計畫 預計 2030 在 南歐 充電樁 市場 達到 15 市佔率 營收 將 會 是 2022 年 的 15 倍',
 '檢測 試劑廠 瑞基 4171TW  今 15 日 表示 因應 未來 事業 方向 進行 內部 組織 升級 明年 將 朝 三 大 方向 發展 包括 人 病 新冠 檢測 畜產 與 伴侶 動物 以及 精準 醫療 藉 此 擴大 事業 版圖 帶動 明年 營運 正向 發展']

# text mining

In [79]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

## 中文TF-IDF

`token_pattern='\\b\\w+\\b'` : 讓TF-IDF認得單字詞

`lowercase=False`

In [84]:
!pip3 install jieba

In [85]:
import jieba
import jieba.analyse

In [86]:
tags = jieba.analyse.extract_tags(corpus_white_join, topK=10)

AttributeError: ignored

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [88]:
text_cv = CountVectorizer(max_df=0.7, min_df=5)

td_matrix = text_cv.fit_transform(corpus_white_join)

In [89]:
tfidf = TfidfTransformer()

tfidf_matrix = tfidf.fit_transform(td_matrix)

In [90]:
topk_arr = np.argsort(-tfidf_matrix.toarray(), axis=1)[:,0:5]

In [ ]:
indices = np.nonzero(-tfidf_matrix.toarray())[0]
#indices[np.argsort(-tfidf_matrix.toarray()[indices])][:,0:5]

In [96]:
corpus_white_join[20:30]

['統包 工程 承攬商 中鼎 9933TW  集團 攜手 欣陸 3703TW 旗下 欣達 環工 執行 桃園市 中壢 地區 污水 下水道 系統 建設 BOT 計畫 今 15 日 就 水 資源 回收 中心 舉行 開工 動土',
 '封測 龍頭 日月光 投控 3711TW 今 15 日 宣布 連續 第六 年 整體 成績 獲 道瓊 永續 指數 半導體 及 半導體 設備 產業 產業 最 高分 為 台灣 首 家 連續 六 度 獲得 產業 最 高分 的 公司',
 '漢磊 3707TW 今 15 日 表示 SiC 產線 第四 季 全數 滿載 整體 毛利率 將 持續 向 上 進一步 提升 至 1518 看好 明年 仍 產能 供不應求 全 年 營收 成長 2 成 起跳 毛利率 也 將 續揚',
 '國產署 近期 與 高雄 屏東 兩 縣市 地方 政府 合作 辦理 2 件 國有 土地 開發案 預計 可 吸引 民間 投資 60億 元 創造 415億 元 產值 與 1422 個 就業 機會',
 '國銀 人民幣 存款 連 3 減 個人戶 連 8 個 月 下滑',
 'PMOLED 廠 錸寶 8104TW 今 15 日 召開 法說 展望 第四 季 錸寶 表示 PMOLED 業務 受 整體 供應鏈 缺料 歐美 塞港 影響 客戶 下單 轉趨 保守 估 本 季 相關 業務 將 面臨 修正 電池 模組 業務 則 在 上游 材料 提前 備貨 下 可望 保持 成長',
 '聯強 2347TW 今 15 日 舉行 法說 副總裁 杜書全 表示 雖然 資訊 商用 成長 力道 放緩 不過 資訊 商用 與 ic 元件 業務 則 受惠 企業 數位 轉型 資料 中心 伺服器 缺料 舒緩 業績 將 明顯 成長 法人 看好 本業 每 股 純益 衝破 6 元',
 '偉詮電 2436TW 今 15 日 公布 10 月 自結 稅 後 純益 053億 元 年增 179 每 股 稅 後 純益 03 元 累計 前 10 月 稅 後 純益 62億 元 年增 204 每 股 稅 後 純益 35 元 受惠 USBPD 出貨 續 旺 加上 10 月 開始 漲',
 '嘉晶 3016TW 今 15 日 表示 部分 客戶 訂單 已 看 至 明年 底 明年 將 持續 調漲 價格 對 展望 樂觀 看待 營收 將 有 雙位數 成長 毛利 也 將 續 

In [95]:
text_cv.get_feature_names_out()[topk_arr][20:30]

array([['承攬商', 'bot', '3703tw', '9933tw', '欣陸'],
       ['產業', '連續', '半導體', '第六', '3711tw'],
       ['毛利率', '續揚', '3707tw', '漢磊', 'sic'],
       ['國有', '屏東', '國產署', '60億', '民間'],
       ['存款', '國銀', '人民幣', '下滑', '001'],
       ['錸寶', 'pmoled', '業務', '8104tw', '轉趨'],
       ['商用', '資訊', '副總裁', '聯強', '2347tw'],
       ['純益', '10', 'usbpd', '62億', '2436tw'],
       ['嘉晶', '明年', '化合物', '毛利', '元宇宙'],
       ['國賓', '飯店', '簽約', '合建', '改建']], dtype=object)

In [94]:

np.array(text_cv.get_feature_names_out())[importance][0:100]

NameError: ignored

In [136]:
text_cv.get_feature_names_out()

array(['001', '003', '004', ..., '龍頭廠', '龐大', '龔明鑫'], dtype=object)